<a href="https://colab.research.google.com/github/ryane-jon/Natural-Language-Processing-Coursework/blob/main/NLP_Coursework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Natural Language Proccessing (CE4145) Coursework
- Ryan Jones (2208751)
- Dataset Source: [Sentiment Labelled Sentences](https://archive.ics.uci.edu/dataset/331/sentiment+labelled+sentences)

I plan to use various NLP techniques to create a model that can predict the sentiment (Positive or Negative) of various sentences. I also plan to extend this to detect the source of these sentences (Amazon, imdb or yelp)

### Import dataset
(from the download of the dataset in the provided link)

The 3 files are imported (amazon_cells_labelled.txt, imdb_labelled.txt, yelp_labelled.txt)

In [2]:
#used to upload data
import io
from google.colab import files
uploaded = files.upload() #files can be uploaded here, I select the 3 labelled files.

Saving amazon_cells_labelled.txt to amazon_cells_labelled.txt
Saving imdb_labelled.txt to imdb_labelled.txt
Saving yelp_labelled.txt to yelp_labelled.txt


In [34]:
#import stuff to use later
from sklearn.pipeline import Pipeline #let's import the pipeline functionality
from sklearn.feature_extraction.text import CountVectorizer #and we will import a simple pre-processing method
from sklearn.feature_extraction.text import TfidfTransformer #and a representation learner
from sklearn.neighbors import KNeighborsClassifier #and a simple classifier model
from sklearn.model_selection import StratifiedKFold #cross fold is sometimes called k-fold. Calling the stratified version ensures that classes have equal representation across folds
from sklearn.metrics import accuracy_score #import an accuracy metric to tell us how well the model is doing

Split each files lines into strings in arrays to be processed

In [138]:
amazon_data = uploaded['amazon_cells_labelled.txt'].decode('UTF-8').splitlines()
imdb_data = uploaded['imdb_labelled.txt'].decode('UTF-8').splitlines()
yelp_data = uploaded['yelp_labelled.txt'].decode('UTF-8').splitlines()

#View some of the data
for index in range(5):
  print(amazon_data[index])
  print(imdb_data[index])
  print(yelp_data[index])

So there is no way for me to plug it in here in the US unless I go by a converter.	0
A very, very, very slow-moving, aimless movie about a distressed, drifting young man.  	0
Wow... Loved this place.	1
Good case, Excellent value.	1
Not sure who was more lost - the flat characters or the audience, nearly half of whom walked out.  	0
Crust is not good.	0
Great for the jawbone.	1
Attempting artiness with black & white and clever camera angles, the movie disappointed - became even more ridiculous - as the acting was poor and the plot and lines almost non-existent.  	0
Not tasty and the texture was just nasty.	0
Tied to charger for conversations lasting more than 45 minutes.MAJOR PROBLEMS!!	0
Very little music or anything to speak of.  	0
Stopped by during the late May bank holiday off Rick Steve recommendation and loved it.	1
The mic is great.	1
The best scene in the movie was when Gerardo is trying to find a song that keeps running through his head.  	1
The selection on the menu was great

Create a numpy array with 3 collumns: Review, Sentiment, and Source

In [139]:
import numpy as np
reviews = []
sentiments = []
sources = []



for line in amazon_data:
  reviews.append(line[:-1])
  sentiments.append(line[-1])
  sources.append("Amazon")

for line in imdb_data:
  reviews.append(line[:-1])
  sentiments.append(line[-1])
  sources.append("IMDB")

for line in yelp_data:
  reviews.append(line[:-1])
  sentiments.append(line[-1])
  sources.append("Yelp")


data = np.column_stack((reviews, sentiments, sources))
print(data.shape)

(3002, 3)


Check for stinky datatype issue

In [140]:
def checkSentimentValid(datas):
  x,y,z = datas.T
  markedForDeath = []
  count = 0
  #for each sentiment value make sure it's "0" or "1"
  for sentiment in y:
    if sentiment != "0" and sentiment != "1":
      ##if not, get as relevant surrounding info
      markedForDeath.append(count)  #an omen
      print(str(count))
      print(x[count]+"\t"+y[count]+"\t"+z[count])
      print(x[count+1]+"\t"+y[count+1]+"\t"+z[count+1]+"\n")
    count= count+1
  return markedForDeath

checkSentimentValid(data)


1178
The script i	s	IMDB
was there a script?  		0	IMDB

1968
Definitely worth seein	g	IMDB
 it's the sort of thought provoking film that forces you to question your own threshold of loneliness.  		1	IMDB



[1178, 1968]

This is a result of reviews containing a character that is misinterpereted as a new line, splitting the review in two

In [141]:
marked = checkSentimentValid(data)
marked.sort(reverse=True)
for marker in marked:
  if sentiments[marker] != "0" and sentiments[marker] != "1":
    reviews[marker+1] = reviews[marker] + sentiments[marker]+" " + reviews[1969] #rejoin the split review as one
    reviews.pop(marker) #remove the leftover data
    sentiments.pop(marker)
    sources.pop(marker)

data = np.column_stack((reviews, sentiments, sources))

1178
The script i	s	IMDB
was there a script?  		0	IMDB

1968
Definitely worth seein	g	IMDB
 it's the sort of thought provoking film that forces you to question your own threshold of loneliness.  		1	IMDB



should now be no issue

In [142]:
checkSentimentValid(data)

[]

can now make a shuffled numpy array

In [147]:
random_array = []
for i in range(len(reviews)):
  random_array.append(i)
np.random.shuffle(random_array)

shuffled_reviews = []
shuffled_sentiments = []
shuffled_sources = []

for i in random_array:
  shuffled_reviews.append(reviews[i])
  shuffled_sentiments.append(sentiments[i])
  shuffled_sources.append(sources[i])

data = np.column_stack((shuffled_reviews, shuffled_sentiments, shuffled_sources))

## Test base NLP Pipeline.
- To start with, it's just predicting the sentiment based on the review text, I will bring the source in (possibly as an additional feature or as a target).
- This section is mostly me testing some pre-proccessing and tokenization methods to see what is actually helpful

In [148]:
x, y, z = data.T  #x=reviews, y=sentiment, z=source
text_clf = Pipeline([ #Pipeline to organise functions
  ('prep', CountVectorizer()), #Count vectorizer
  ('rep', TfidfTransformer()), #representation learning method using tf-idf
  ('mod', KNeighborsClassifier()), #kNN classifier
  ])

acc_score = []

kf = StratifiedKFold(n_splits=5)
for train, test in kf.split(x,y):
  x_train, x_test, y_train, y_test = x[train], x[test], y[train], y[test]
  text_clf.fit(x_train, y_train) #fit to training data
  predictions = text_clf.predict(x_test) #predict on test data
  acc_score.append(accuracy_score(predictions, y_test)) #get accuracy

print("Accuracy:", np.mean(acc_score)) #mean accuracy


Accuracy: 0.7803333333333333


Double-checking that data is being represented correctly

In [149]:
for index in range(0,4):
  print(x[index*100])
  print(y[index*100])
  print(z[index*100])

Both Rickman and Stowe play their roles to the hilt in this tale of a childrens' book writer who-- maybe?-- has written a subversive tract.  	
1
IMDB
It features an outlandish array of memorable, psychotic but lovable nuts.  	
1
IMDB
Oh and I forgot to also mention the weird color effect it has on your phone.	
0
Amazon
Internet is excrutiatingly slow.	
0
Amazon


### Introduce tokenization pre-proccessing into the pipeline

Import packages for tokenization

In [150]:
import nltk #import the natural language toolkit

nltk.download('punkt') #download the package in nltk which supports tokenization
nltk.download('punkt_tab')
nltk.download('stopwords') #download the nltk package for stopwords

from nltk.tokenize import word_tokenize #import the tokenize package
from nltk.corpus import stopwords #import the package from the corpus
from nltk.stem.snowball import SnowballStemmer #import the snowball stemmer (also known as Porter2)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Create a tokenize function and add it to the pipeline

In [151]:
from sklearn.base import BaseEstimator, TransformerMixin
class pre_process_tokenize(BaseEstimator, TransformerMixin):

    def __init__(self):
      return None

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
      prep_text = []
      for x in X:
            #basic tokenisation
            token_text = word_tokenize(x)
            #lower casing & punctiuation removal)
            normd_text = [token.lower() for token in token_text]
            #stopword removal
            swr_text = [token for token in normd_text if token not in stopwords.words('english')] #list compression to remove any stopwords from our list
            #stemming
            stemmer = SnowballStemmer("english") #specify we are using the English stemming rules, as other languages are present in toolkit
            prep_text += [[stemmer.stem(word) for word in swr_text]] #list compression for applying the stemmer

      #rejoin the sentences for use of CountVectorizer
      prep_sentences = [" ".join(sentence) for sentence in prep_text]
      return prep_sentences

text_clf = Pipeline([
  ('prep', pre_process_tokenize()),
  ('count', CountVectorizer()),
  ('rep', TfidfTransformer()),
  ('mod', KNeighborsClassifier()),
  ])


acc_score = []

kf = StratifiedKFold(n_splits=5)
for train, test in kf.split(x,y):

  x_train, x_test, y_train, y_test = x[train], x[test], y[train], y[test]

  text_clf.fit(x_train, y_train)
  predictions = text_clf.predict(x_test)
  acc = accuracy_score(predictions, y_test)
  acc_score.append(acc)

print("Accuracy:", np.mean(acc_score))

Accuracy: 0.7496666666666667


Tokenisation seems to have reduced the accuracy of the model (which I personally didn't expect).


It's possible that stemming and stop word removal caused some context and nuance to be lost, which was made worse by the fact that the data comes from multiple different sources. I.E Without the source to contextualise the review, the feature starts to lose some important meaning.

In [152]:
class pre_process_tokenize2(BaseEstimator, TransformerMixin):

    def __init__(self):
      return None

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
      prep_text = []
      for x in X:
            #basic tokenisation
            token_text = word_tokenize(x)
            #lower casing & punctiuation removal)
            normd_text = [token.lower() for token in token_text]
            #stemming
            stemmer = SnowballStemmer("english") #specify we are using the English stemming rules, as other languages are present in toolkit
            prep_text += [[stemmer.stem(word) for word in normd_text]] #list compression for applying the stemmer

      #rejoin the sentences for use of CountVectorizer
      prep_sentences = [" ".join(sentence) for sentence in prep_text]
      return prep_sentences

text_clf = Pipeline([
  ('prep', pre_process_tokenize2()),
  ('count', CountVectorizer()),
  ('rep', TfidfTransformer()),
  ('mod', KNeighborsClassifier()),
  ])


acc_score = []

kf = StratifiedKFold(n_splits=5)
for train, test in kf.split(x,y):

  x_train, x_test, y_train, y_test = x[train], x[test], y[train], y[test]

  text_clf.fit(x_train, y_train)
  predictions = text_clf.predict(x_test)
  acc = accuracy_score(predictions, y_test)
  acc_score.append(acc)

print("Accuracy:", np.mean(acc_score))

Accuracy: 0.7836666666666667


Testing for different variations of tokenization pre-proccess (just one test so not super accurate, gives a general idea)
- Tokenisation with **lower casing and punctiuation removal** only: 76.2%
- Tokenisation with **lower casing, punctiuation removal** and **stemming**: 76.6%
- Tokenisation with **lower casing, punctiuation removal** and **stopword removal**: 74.6%
- Tokenisation with **stemming** only: 76.6%

**Using only the CountVectorizer yielded 76.7%**, so the tokenization step is being abandoned for now, though I expect it could be more useful in predicting sources.

In [136]:
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical



In [137]:
def mlp(dataset_size, num_classes):
  input_shape = (dataset_size,)

  model = Sequential()
  model.add(Dense(128, input_shape=input_shape, activation='relu'))
  model.add(Dense(128, activation='relu'))
  model.add(Dense(num_classes, activation='softmax')) #remember this format from last year? We effectively parameterse two hidden layers and one output layer
  return model

from sklearn.feature_extraction.text import TfidfVectorizer #we will import the tfidf functionality from sklearn

tfidf_base = TfidfVectorizer(max_features=1000) #we can instantiate, but we will need to recreate every fold in our evaluation (to prevent test set leakage)

In [135]:
kf = StratifiedKFold(n_splits=5, shuffle=True)

# Re-initialize acc_score
acc_score = []

for train, test in kf.split(x,y):
  x_train, x_test, y_train, y_test = x[train], x[test], y[train], y[test]
  tfidf = tfidf_base
  x_train = tfidf.fit_transform(x_train)
  x_train = x_train.todense() #by default, tfidf will output a sparse matris to conserve memory. This is incompatible with our deep learner
  x_test = tfidf.transform(x_test)
  x_test = x_test.todense()

  model = mlp(1000, 2) #we also instantiate a new mlp to prevent leakage of train and test set
  y_train = to_categorical(y_train, 2) #convert y to one hot vectors
  y_test = to_categorical(y_test, 2)

  # Configure the model and start training
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy']) #we have parameterised with fairly standard metrics - do feel free to alter and investigate
  model.fit(x_train, y_train, epochs=10, batch_size=250, verbose=1, validation_split=0.2)

  # Test the model after training
  test_results = model.evaluate(x_test, y_test, verbose=1)
  print(f'Test results - Loss: {test_results[0]} - Accuracy: {test_results[1]}%')

  acc_score.append(test_results[1])

print("Accuracy:", np.mean(acc_score))

Epoch 1/10


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


8/8 ━━━━━━━━━━━━━━━━━━━━ 2s 52ms/step - accuracy: 0.5260 - loss: 0.6926 - val_accuracy: 0.6479 - val_loss: 0.6885
Epoch 2/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.7667 - loss: 0.6835 - val_accuracy: 0.7563 - val_loss: 0.6785
Epoch 3/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.8466 - loss: 0.6661 - val_accuracy: 0.7854 - val_loss: 0.6560
Epoch 4/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.8666 - loss: 0.6277 - val_accuracy: 0.8000 - val_loss: 0.6118
Epoch 5/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.8729 - loss: 0.5571 - val_accuracy: 0.8104 - val_loss: 0.5437
Epoch 6/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.8970 - loss: 0.4549 - val_accuracy: 0.8083 - val_loss: 0.4744
Epoch 7/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.9038 - loss: 0.3431 - val_accuracy: 0.8042 - val_loss: 0.4299
Epoch 8/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.9149 - loss: 0.2636 - val_accuracy: 0.8083 - val_loss: 0.4196
Epoch 9/10
